In [1]:
# %pip install pandas
# %pip install matplotlib
# %pip install scikit-learn
# %pip install category_encoders
# %pip install xgboost

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
db = pd.read_csv('C:\\Users\\DOR CO\\Desktop\\Real state ML\\Real-Estate-Price-Estimator\\gateaway\\properties.csv')
x = db.drop(columns=['price'])
y = db['price']

In [4]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(transformers=[
    ('target', ce.TargetEncoder(cols=["Province", "City"]), ["Province", "City"]),
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ["property_type"]),
    ('scaler', StandardScaler(), ['property_quality','rooms','living_area','bathrooms','garden_sqm',
                                  'terrace_sqm','land_area','distance_from_airport','Skiresort_distance',
                                  'terrace','garden','pool','car_box'])
], remainder='drop')


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline

models = {
    "Linear": LinearRegression(),
    "Polynomial (deg=2)": Pipeline([("poly", PolynomialFeatures(degree=2)), ("lin", LinearRegression())]),
    "SVR": SVR(kernel='rbf'),
    "DecisionTree": DecisionTreeRegressor(random_state=0),
    "RandomForest": RandomForestRegressor(n_estimators=200, random_state=0),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=200, random_state=0),
    "XGBoost": XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=0)
}

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import r2_score
r_scores = {}
kf = KFold(n_splits=5, shuffle=True, random_state=0)
for name, model in models.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    neg_mse_scores = cross_val_score(pipeline, x, y, cv=kf, scoring='neg_mean_squared_error')
    r_scores[name] = r2_score(y_test, y_pred)
best_r2 = max(r_scores.values())
percentages = {model: (best_r2 / r2) * 100 for model, r2 in r_scores.items()}
best_model = max(percentages, key=percentages.get)
print(f"Best model: {best_model} with {percentages[best_model]:.2f}%")

Best model: Linear with 100.00%


{'Linear': np.float64(4134485.6906023575),
 'Polynomial (deg=2)': np.float64(4341165.763539091),
 'SVR': np.float64(4281470.948206601),
 'DecisionTree': np.float64(5232475.358623042),
 'RandomForest': np.float64(4368288.264359077),
 'GradientBoosting': np.float64(4683761.172422262),
 'XGBoost': np.float64(4652310.38370233)}